# Headers 

In [ ]:
from theano.sandbox import cuda
cuda.use('gpu0')
import os, cv2, sys
import utils; reload(utils)
from utils import *
from shutil import copyfile
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras import backend as K
K.set_image_dim_ordering('th')

from keras.utils import np_utils
from keras import regularizers
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Convolution2D, MaxPooling2D
# from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import *
from keras.optimizers import SGD,RMSprop,Adam

from sklearn.metrics import classification_report,confusion_matrix
import itertools

## Setting Data Path Directory

* some configurations for convolutional neuralnet

In [ ]:
%cd ~/work/deeplearning/
batch_size=16

path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"

train_path = path + "/train"
valid_path = path + "/valid"
test_path = path + "/test"

img_rows=128
img_cols=128
num_channel=1
num_epoch=60
num_classes = 2

batch = []
batch_accuracy = []
batch_loss = []

names = ['cats','dogs']

* Following functions copies(removes existing data first) different sizes of train/validation set for further processing

In [ ]:
#clean previous data
def clean_up():
    %rm $path/train/cats/*
    %rm $path/train/dogs/*
    %rm $path/valid/cats/*
    %rm $path/valid/dogs/*

#copy training images
def copy_samples(train, validation):
    %cd ../data/2cat/train
    #clean previous data
    clean_up()
    
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(train): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd ~/work/deeplearning/

* Following function takes the input as path to the data, and returns grey-scaled images and labels for those images.

In [ ]:
def get_greyed_images(data_path):
    data_dir_list = os.listdir(data_path)
    img_data_list=[]
    labels=[]

    for dataset in data_dir_list:
        img_list=os.listdir(data_path+'/'+ dataset)
        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
        for img in img_list:

            if 'cat' in img:
                labels.append(0)
            else:
                labels.append(1)


            input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
            input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
            input_img_resize=cv2.resize(input_img,(128,128))
            img_data_list.append(input_img_resize)

    img_data = np.array(img_data_list)
    img_data = img_data.astype('float32')
    img_data /= 255
    print (img_data.shape) 
    return img_data, labels

In [ ]:
def get_numpy_images(data_path):
    data_dir_list = os.listdir(data_path)
    img_data_list=[]
    labels=[]

    for dataset in data_dir_list:
        img_list=os.listdir(data_path+'/'+ dataset)
        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
        for img in img_list:

            if 'cat' in img:
                labels.append(0)
            else:
                labels.append(1)

            img_data_list.append(input_img_resize)

    img_data = np.array(img_data_list)
    img_data = img_data.astype('float32')
    return img_data, labels

#### Load Training, Validation, and Test Data

In [ ]:
def setup_data(train_size, valid_size):
    copy_samples(train_size, valid_size)
    
#     #Training Data
#     train_img_data, train_labels = get_greyed_images(train_path)
#     #expand a dimension to add 1 to the shape of train_img_data
#     train_img_data= np.expand_dims(train_img_data, axis=1) 
#     train_nsamples = train_img_data.shape[0]
#     train_input_shape=train_img_data[0].shape

#     train_Y = np_utils.to_categorical(train_labels, num_classes)

#     #Validation Data

#     valid_img_data, valid_labels = get_greyed_images(valid_path)
#     #expand a dimension to add 1 to the shape of train_img_data
#     valid_img_data= np.expand_dims(valid_img_data, axis=1) 
#     valid_nsamples = valid_img_data.shape[0]
#     valid_input_shape=valid_img_data[0].shape

#     valid_Y = np_utils.to_categorical(valid_labels, num_classes)


    #Test Data
    test_img_data, test_labels = get_greyed_images(test_path)
    test_img_data = np.expand_dims(test_img_data, axis=1)
    test_nsamples = test_img_data.shape[0]
    test_input_shape=test_img_data[0].shape

    test_Y = np_utils.to_categorical(test_labels, num_classes)

## Build Model on Training Set

* Following function defines a model

In [ ]:
def get_cnn_model(input_shape):
    model = Sequential()

    #model.add(BatchNormalization(axis=1, input_shape=input_shape))
    model.add(Convolution2D(32, 3,3,border_mode='same', input_shape=input_shape))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(64, 3, 3, activity_regularizer=regularizers.l2(0.01)))
    #model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.3))

    model.add(Convolution2D(100, 3, 3, activity_regularizer=regularizers.l2(0.01)))
    #model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    #model.add(Convolution2D(64, 3, 3))
    #model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(Dropout(0.3))

    model.add(Convolution2D(32, 3, 3, activity_regularizer=regularizers.l2(0.01)))
    #model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    #model.add(Convolution2D(64, 3, 3))
    #model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.3))
    
#     model.add(Convolution2D(32, 3, 3))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.5))    
    
    model.add(Flatten())
    model.add(Dense(200,  activity_regularizer=regularizers.l2(0.01)))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.3))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    sgd = SGD(lr=0.001, momentum=0.2, nesterov=True)
    adm = Adam()
    #model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])  
    
    return model

In [ ]:
def get_cnn_II(train_batches, val_batches, input_shape):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=input_shape),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
#     model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
#                      nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=50, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

* Following code sets up the data and build a model

### Make Predictions

In [ ]:
num_epoch = 60
train_size = 18000
valid_size = int(math.floor(.2 * train_size))
setup_data(train_size, valid_size)
model = get_cnn_model(train_input_shape)
#model.summary()
hist = model.fit(train_img_data, train_Y, batch_size=16, nb_epoch=num_epoch, verbose=1, validation_data=(valid_img_data, valid_Y))
model.save_weights(results_path + '/ft_'+ str(len(train_img_data))+ '.e' + str(num_epoch))

In [ ]:
model.load_weights(results_path + '/ft_' + str(len(train_img_data))+ '.e' + str(num_epoch))
score = model.evaluate(test_img_data, test_Y, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

# batch.append(len(train_img_data))
# batch_accuracy.append(score[1])
# batch_loss.append(score[0])

train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']


generate_graph(1, range(num_epoch), train_loss, val_loss, 'Loss', 'Train Loss', 'Validation Loss', train_size)
generate_graph(2, range(num_epoch), train_acc, val_acc, 'Accuracy', 'Train Accuracy', 'Validation Accuracy', train_size)



### Generating graphs

* Following functions generate graphs

In [ ]:
def generate_graph(fig_no, epochs, train, val, label, train_title, val_title, train_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(epochs,train)
    plt.plot(epochs,val)
    plt.xlabel('num of Epochs')
    plt.ylabel(label)
    plt.title(train_title + ' vs ' + val_title + '( Samples:' + str(train_size) + ')')
    plt.grid(True)
    plt.legend(['train','val'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(results_path + '/batch_graphs/' +  label + '_' + str(train_size) + '.jpg')

In [ ]:
Y_pred = model.predict(test_img_data)
print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
#y_pred = model.predict_classes(X_test)
#print(y_pred)
target_names = ['class 0(cats)', 'class 1(Dogs)']

print(classification_report(np.argmax(test_Y,axis=1), y_pred,target_names=target_names))

print(confusion_matrix(np.argmax(test_Y,axis=1), y_pred))

In [ ]:
#y_pred[:20]
#np.argmax(test_Y, axis=1)[:20]

correct = np.where(y_pred == np.argmax(test_Y, axis=1))
accuracy = float(len(correct[0]))/float(len(test_Y))
accuracy

##### Single Prediction

In [ ]:
test_path

In [ ]:
test_image = cv2.imread(test_path + '/cats/cat.6296.jpg')
test_image=cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
test_image=cv2.resize(test_image,(128,128))
test_image = np.array(test_image)
test_image = test_image.astype('float32')
test_image /= 255


test_image= np.expand_dims(test_image, axis=0)
test_image= np.expand_dims(test_image, axis=0)

print((model.predict(test_image)))

In [ ]:
#fig=plt.figure(figsize=(16,16))

type(test_image)

#plt.imshow(test_image[:,:,0],cmap='gray')

In [ ]:
cv2.imshow('image',test_image)